In [2]:
import requests
from tqdm import tqdm

In [3]:
# Идентификатор компании
sber = '3529'
# Номер страницы
page = 1
# Количество вакансий на страницу
num_per_page = 100
# Идентификатор области
moscow = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'

In [4]:
# Отправляем GET запрос в api.hh.ru
res = requests.get(url)
# Преобразовываем ответ api из JSON в словарь dict()
vacancies = res.json()
# Получаем количество страниц по результатам запроса
num_pages = vacancies.get('pages')
num_pages

16

In [5]:
# Вывод ключей словаря
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

In [11]:
vacancies['items'][0]

{'id': '89084783',
 'premium': False,
 'name': 'Разработчик SQL',
 'department': {'id': '3529-3529-data', 'name': 'Сбер. Data Science'},
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2023-12-08T09:13:27+0300',
 'created_at': '2023-12-08T09:13:27+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=89084783',
 'branding': {'type': 'MAKEUP', 'tariff': None},
 'show_logo_in_search': True,
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/89084783?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/89084783',
 'relations': [],
 'employer': {'id': '3529',
  'name': 'СБЕР',
  'url': 'https://api.hh.ru/employers/3529',
  'alternate_url': 'https://hh.ru/employer/3529',
  'logo_urls': {'240': 'ht

In [6]:
# Получаем список идентификаторов вакансий из короткого описания вакансий
# с одной страницы
vacancy_ids = [el.get('id') for el in vacancies.get('items')]

In [7]:
len(vacancy_ids)

100

In [13]:
# Запускаем цикл для сбора всех идентификаторов вакансий со всех страниц
all_vacancy_ids= []
for i in tqdm(range(num_pages)):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    # заполняем список элементами vacancy_ids с помощью метода extend
    all_vacancy_ids.extend(vacancy_ids)

100%|██████████| 16/16 [00:16<00:00,  1.00s/it]


In [14]:
# Проверка количества идентификаторов вакансий
len(all_vacancy_ids)

1562

In [15]:
# Получение полной информации о вакансии по идентификатору
# Отправляем запрос с идентификатором вакансии
vac_id = 90572059
url = f'https://api.hh.ru/vacancies/{vac_id}'
res = requests.get(url)
vacancy = res.json()

In [ ]:
# Пример полного описания вакансии
vacancy

In [17]:
# Собираем массив с полными описаниями вакансий по идентификаторам
full_vacancies = []
for vac_id in tqdm(all_vacancy_ids):
    url = f'https://api.hh.ru/vacancies/{vac_id}'
    res = requests.get(url)
    vacancy = res.json()
    full_vacancies.append(vacancy)

100%|██████████| 1562/1562 [14:18<00:00,  1.82it/s]


In [18]:
# Создаем таблицу DataFrame из всех полученных данных
import pandas as pd
df = pd.json_normalize(full_vacancies)
df.head()

,id,premium,relations,name,insider_interview,response_letter_required,salary,allow_messages,contacts,description,...,employer.trusted,salary.from,salary.to,salary.currency,salary.gross,address.metro,address,test.required,immediate_redirect_url,immediate_redirect_vacancy_id
0,90591314,False,[],HR специалист (командировки и льготы),None,True,NaN,True,None,<p>Мы находимся в поисках специалиста hr коман...,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88818100,False,[],Старший менеджер по работе с ключевыми клиентами,None,False,NaN,True,None,<p>Каждый второй предприниматель малого и сред...,...,True,195500.0,NaN,RUR,True,NaN,NaN,NaN,NaN,NaN
2,90433682,False,[],Специалист по документообороту,None,False,NaN,False,None,<p>Сбер — это 14 тысяч подразделений по всей с...,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,87517053,False,[],Старший клиентский менеджер,None,False,NaN,False,None,<p><strong>Старший клиентский менеджер</strong...,...,True,115000.0,NaN,RUR,True,NaN,NaN,NaN,NaN,NaN
4,89692763,False,[],Стажер в команду по технорискам,None,False,NaN,False,None,<p>Мы Команда Сбера по технорискам. Не без гор...,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.shape

(1562, 86)

In [21]:
# Сохраняем результаты
# Возможно потребуется установка модуля openpyxl
# !pip install openpyxl

# Сохраняем в csv файл
df.to_csv('vacancies.csv', index=False)
# Сохраняем в эксель-файл
df.to_excel('vacancies.xlsx', index=False)
# Сохраняем в файл в формате pickle
df.to_pickle('vacancies.pkl')